In [9]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.client import timeline
from datetime import datetime
import network as vgg

def get_init_restorer():
    variables_to_restore = []
    # for var in slim.get_model_variables():
    for var in tf.global_variables():
        variables_to_restore.append(var)
    
    return tf.train.Saver(variables_to_restore)

scale_size = 224
cache_folder = '/export/home/qliu24/qing_voting_139/qing_voting_py/cache/'
checkpoints_dir = os.path.join(cache_folder, 'checkpoints_vgg')
tf.logging.set_verbosity(tf.logging.INFO)
with tf.device('/cpu:0'):
    input_images = tf.placeholder(tf.float32, [1, None, None, 3])

vgg_var_scope = 'vgg_16'
with tf.variable_scope(vgg_var_scope, reuse=False):
    with slim.arg_scope(vgg.vgg_arg_scope()):
        _, end_points = vgg.vgg_16(input_images)
        
restorer = get_init_restorer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
init_op = tf.global_variables_initializer()
# Run the session:
sess = tf.Session(config=config)
# init
print(str(datetime.now()) + ': Start Init')

restorer.restore(sess, os.path.join(checkpoints_dir, 'vgg_16.ckpt'))

print(str(datetime.now()) + ': Finish Init')

2017-07-31 18:41:42.905937: Start Init
INFO:tensorflow:Restoring parameters from /export/home/qliu24/qing_voting_139/qing_voting_py/cache/checkpoints_vgg/vgg_16.ckpt
2017-07-31 18:41:44.336548: Finish Init


In [6]:
import math
import pickle
from scipy.spatial.distance import cdist
import scipy.io as sio
from myresize import myresize
import numpy as np
import sys
import os
import cv2

adv_unv_file = '/export/home/qliu24/VC_adv_data/qing/VGG_unv/VGG-16.mat'
matcontent = sio.loadmat(adv_unv_file)
r = matcontent['r']
print(r.shape, np.mean(np.absolute(r)), np.max(r), np.min(r))

scale_size = 224
featDim = 512
img_mean = np.array([104., 117., 124.]).reshape(1,1,3)

category = 'car'
target = 'bus'

dataset_suffix = 'mergelist_rand'
Dataset = dict()
Dataset['img_dir'] = '/export/home/qliu24/dataset/PASCAL3D+_release1.1/Images/{0}_imagenet/'
Dataset['anno_dir'] = '/export/home/qliu24/dataset/PASCAL3D+_release1.1/Annotations/{0}_imagenet/'
Dataset['gt_dir'] = '/export/home/qliu24/qing_voting_139/qing_voting_py/intermediate/ground_truth_data/'
Dataset['train_list'] = os.path.join(Dataset['gt_dir'], '{0}_'+ '{0}_train.txt'.format(dataset_suffix))
Dataset['test_list'] = os.path.join(Dataset['gt_dir'], '{0}_'+ '{0}_test.txt'.format(dataset_suffix))
Dataset['adv_dir'] = '/export/home/qliu24/VC_adv_data/qing/VGG_adv/{}_{}'

dir_img = Dataset['img_dir'].format(category)
file_list = Dataset['test_list'].format(category)

with open(file_list, 'r') as fh:
    content = fh.readlines()
    
img_list = [x.strip().split() for x in content]
img_num = len(img_list)
print('total number of images: {0}'.format(img_num))



(224, 224, 3) 7.73181 10.0 -10.0
total number of images: 1000


In [7]:
img_ori=[]
for nn in range(img_num):
#     file_adv = os.path.join(dir_adv, '{0}.pickle'.format(img_list[nn][0]))
#     assert(os.path.isfile(file_adv))
#     with open(file_adv, 'rb') as fh:
#         im_ori, _, im_fool = pickle.load(fh)
    
    file_img = os.path.join(dir_img, '{0}.JPEG'.format(img_list[nn][0]))
    assert(os.path.isfile(file_img))
    im_ori = cv2.imread(file_img)
    # im_ori = myresize(im_ori, scale_size, 'short')
    im_ori = cv2.resize(im_ori, (scale_size, scale_size))
    
    img_ori.append(im_ori)
    
# print(np.mean(r_ls1), np.min(r_ls1), np.max(r_ls1))
# print(np.mean(r_ls2), np.min(r_ls2), np.max(r_ls2))
print(len(img_ori))

1000


In [11]:
label_ls = []
prob_ls = []
# r = np.expand_dims(r, axis=0)[:,:,:,[2,1,0]]
for nn in range(img_num):
    if nn%50==0:
        print(nn, end=' ', flush=True)
        
    img = np.copy(img_ori[nn]).astype('float64')
    img -= img_mean
    img = np.expand_dims(img, axis=0)
    scores = sess.run(end_points['vgg_16/fc8/reduced'], feed_dict={input_images: img})[0]
    probs_ori = np.exp(scores)/np.sum(np.exp(scores))
    
    img += r
    scores = sess.run(end_points['vgg_16/fc8/reduced'], feed_dict={input_images: img})[0]
    probs_fool = np.exp(scores)/np.sum(np.exp(scores))
    
    label_ls.append((np.argmax(probs_ori), np.argmax(probs_fool)))
    prob_ls.append((np.max(probs_ori), np.max(probs_fool)))

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 

In [13]:
label_ls = np.array(label_ls).astype(int)
prob_ls = np.array(prob_ls)

In [16]:
car_labels = [436, 468, 511, 609, 627, 656, 717, 734, 751, 817]
print(np.sum([label_ls[nn,0] == label_ls[nn,1]for nn in range(label_ls.shape[0])]))
print(np.sum([label_ls[nn,0] in car_labels for nn in range(label_ls.shape[0])]))
print(np.sum([label_ls[nn,1] in car_labels for nn in range(label_ls.shape[0])]))

275
902
389
